In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from konlpy.tag import Hannanum, Mecab, Kkma, Komoran, Okt
import time

In [2]:
csv_path = os.getenv("HOME") + "/aiffel/news_crawler/news_data3.csv"
df = pd.read_table(csv_path, sep=',')
df.sample(10)

,news,code
1751,그래픽=이스트시큐리티 알약 블로그 캡처\n\n[이데일리 이후섭 기자] 랜섬웨어는 불...,IT/과학
906,/사진=김철민 페이스북 /사진=김철민 페이스북\n\n개그맨 김철민이 5월 공연을 예...,생활/문화
1060,문제는 결국 돈이다. '게임이용장애' 질병 등록 시도는 게임업계에서 돈을 거둬들이기...,IT/과학
1164,[디지털데일리 이종현기자] 스마트헬스케어 기업 인성정보는 신종코로나바이러스감염증-1...,IT/과학
926,트위터는 매년 두 차례 ‘해크위크(hack week)’라는 아이디어 개발 기간을 둔...,생활/문화
804,(자료사진) © AFP=뉴스1 (자료사진) © AFP=뉴스1\n\n(서울=뉴스1) ...,생활/문화
1346,의왕사랑상품권 설명절 특별할인 안내문. 사진제공=의왕시 의왕사랑상품권 설명절 특별할...,사회
1372,(울산=뉴스1) 윤일지 기자 = 서울중앙지검 공공수사2부 수사관들이 4일 오후 울산...,사회
1049,효성인포메이션시스템은 국가정보자원관리원의 정보자원 통합 및 지능형 클라우드 인프라 ...,IT/과학
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회


In [3]:
# 정규 표현식을 이용해 한글만 사용
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

0       파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기자 파주시는 ...
1       동영상 뉴스이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소 공...
2       황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...
3       귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...
4       서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 사회적 거리두기에...
                              ...                        
1753    서울뉴시스 이재은 기자  한국인터넷기업협회인기협는 정보통신서비스 제공자에게 사적 검...
1754    리처드 플레플러 전   연합뉴스 자료사진 리처드 플레플러 전   연합뉴스 자료사진샌...
1755    한국의 인공지능 기술과 산업화 수준이 미국중국영국일본 등 경쟁국보다 뒤지는 것으로 ...
1756    서울뉴시스 오동현 기자  넥슨의 온라인 캐주얼 레이싱게임 크레이지레이싱 카트라이더에...
1757    서울뉴시스 이재은 기자  한국인터넷진흥원은 한국경영인증원으로부터 인권경영시스템 인증...
Name: news, Length: 1758, dtype: object

In [4]:
# 데이터 Null값 확인
print(df.isnull().sum())

news    0
code    0
dtype: int64


In [5]:
# 중복 샘플 제거
df.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df))

뉴스 기사의 개수:  1496


In [6]:
print(df.groupby('code').size().reset_index(name = 'count'))

    code  count
0  IT/과학    458
1     사회    526
2  생활/문화    512


In [7]:
# 불용어 처리
stopwords = ['고','된다','이미','부터','에서','를','하다','한편','으로','각','총','사진','에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']

In [8]:
# 토크나이즈 성능 비교를 위한 토크나이즈 설정
from konlpy.tag import Hannanum, Mecab, Kkma, Komoran, Okt

Mecab_t = Mecab()
Hannanum_t = Hannanum()
Kkma_t = Kkma()
Komoran_t = Komoran(userdic='/tmp/dic.txt')
Okt_t = Okt()

In [9]:
# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수
def preprocessing(tokenizer, data):
  text_data = []

  for sentence in data:
    temp_data = []
    temp_data = tokenizer.morphs(sentence) 
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [10]:
def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

In [11]:
start=time.time()

text_data = preprocessing(Mecab_t, df['news'])

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#- 나이브 베이즈 분류기를 수행
#- X_train은 TF-IDF 벡터, y_train은 레이블
clf = MultinomialNB().fit(X_train_tfidf, y_train)

y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))
print("[Mecab, preprocessing 걸린 시간]", time.time()-start, 'sec')

              precision    recall  f1-score   support

       IT/과학       0.81      0.88      0.85       113
          사회       0.83      0.85      0.84       136
       생활/문화       0.83      0.75      0.79       125

    accuracy                           0.83       374
   macro avg       0.83      0.83      0.83       374
weighted avg       0.83      0.83      0.83       374

[Mecab, preprocessing 걸린 시간] 2.448532819747925 sec


In [12]:
start=time.time()

text_data = preprocessing(Hannanum_t, df['news'])
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))
print("[Hannanum, preprocessing 걸린 시간]", time.time()-start, 'sec')

              precision    recall  f1-score   support

       IT/과학       0.77      0.88      0.82       113
          사회       0.87      0.76      0.81       136
       생활/문화       0.79      0.80      0.80       125

    accuracy                           0.81       374
   macro avg       0.81      0.81      0.81       374
weighted avg       0.82      0.81      0.81       374

[Hannanum, preprocessing 걸린 시간] 48.43423533439636 sec


In [13]:
start=time.time()
text_data = preprocessing(Kkma_t, df['news'])
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))
print("[Kkma, preprocessing 걸린 시간]", time.time()-start, 'sec')

              precision    recall  f1-score   support

       IT/과학       0.82      0.90      0.86       113
          사회       0.84      0.82      0.83       136
       생활/문화       0.82      0.76      0.79       125

    accuracy                           0.83       374
   macro avg       0.83      0.83      0.83       374
weighted avg       0.83      0.83      0.83       374

[Kkma, preprocessing 걸린 시간] 399.4901967048645 sec


In [14]:
start=time.time()

text_data = preprocessing(Komoran_t, df['news'])
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))
print("[Komoran, preprocessing 걸린 시간]", time.time()-start, 'sec')

              precision    recall  f1-score   support

       IT/과학       0.80      0.88      0.84       113
          사회       0.82      0.83      0.83       136
       생활/문화       0.82      0.74      0.78       125

    accuracy                           0.82       374
   macro avg       0.82      0.82      0.82       374
weighted avg       0.82      0.82      0.81       374

[Komoran, preprocessing 걸린 시간] 54.31455898284912 sec


In [15]:
start=time.time()

text_data = preprocessing(Okt_t, df['news'])
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))
print("[Okt, preprocessing 걸린 시간]", time.time()-start, 'sec')

              precision    recall  f1-score   support

       IT/과학       0.81      0.88      0.85       113
          사회       0.82      0.86      0.84       136
       생활/문화       0.84      0.74      0.79       125

    accuracy                           0.83       374
   macro avg       0.83      0.83      0.83       374
weighted avg       0.83      0.83      0.82       374

[Okt, preprocessing 걸린 시간] 45.5942268371582 sec
